In this tutorial, we will cover:

- Convolution operators on non-Euclidean domains

##Import dependencies (run the following cells)

In [ ]:
# @title import/install dependencies

!pip install networkx
!pip install python-igraph

import numpy as np
import torch
from torch.nn import Module
from torch import nn
import torch.nn.functional as F

import torch.optim as optim
import matplotlib.pyplot as plt

from tqdm import trange

from __future__ import print_function, division

In [ ]:
# @title reproducibility stuff

import random

np.random.seed(4)
random.seed(4)

torch.cuda.manual_seed(4)
torch.manual_seed(4)
torch.backends.cudnn.deterministic = (
    True  # Note that this Deterministic mode can have a performance impact
)
torch.backends.cudnn.benchmark = False

# Introduction

## Information embedded in data

In machine learning we build models using the information contained in a collection of samples.

We can look at the information embedded in data as a combination of:
- The **signal**, encoding the features of the variables.
- The **structure**, encoding the relations between variables.

Formally we encode the features of these variables in a function $f$ over a domain $\Omega$ which defines the relations.

![alt text](https://raw.githubusercontent.com/lucmos/DLAI-s2-2020-tutorials/master/09/pics/data-signal-domain.PNG)



Sometimes most of the information is in the signal.
> Consider the problem of predicting the hour of the day from pixels in the [Monet's paintings of the Rouen Cathedral](https://en.wikipedia.org/wiki/Rouen_Cathedral_(Monet_series)). Can you solve this problem without considering the structure?
>
>![alt text](https://raw.githubusercontent.com/lucmos/DLAI-s2-2020-tutorials/master/09/pics/picture.png)
>
> **EXERCISE**: Sketch a learning pipeline to solve this problem without considering the structure.

Sometimes most of the information is in the structure.
> Consider the problem of predicting members of a chess club in a social network. You know the true label of few people (member/not member), and some features for each person, say age and gender. Can you solve this problem without considering the structure?

>Another relevant example are proteins, which are essential elements for growth and repair, good functioning and structure of all living cells. Proteins are macromolecules composed by a chain of  smaller molecules (the amminoacids) which spontanuosly folds in a complex 3D arrangement. The functionality of proteins comes almost entirely from their shapes rather than the specific atoms of their composition.

To learn proper models, we should be able to grasp all the relevant information in our data, whether in the signal or in the structure.

Notice that often the mutual information between the signal and the structure is non-zero, i.e. you may obtain some information about the signal observing only the structure and viceversa. Nevertheless it may be much easier to extract the information from one of the two.


> Consider again the proteins, suppose to have a dataset composed of 3d triangle meshes encoding their shapes and some categorical features encoded as functions defined on each vertex of the mesh, such as the amminoacid in that region of the protein and its positional encoding (the position of the amminoacid in the chain).
>
>This kind of representation is redundant; the information about the shape of the protein is already contained in the chain of amminoacids, since this folds in a deterministic way. Your structure information is already encoded in the signal. In theory you can predict shape-dependent quantities, such as the degree of interaction between different proteins, using only the information in the signal; nevertheless extracting the shape information from the sequence of amminoacids is a very hard problem by itself (known as protein folding, one of the biggest open problems in biology $^1$) and in practice you obtain much better results using the shape information encoded in the triangle mesh. See for instance [*Deciphering interaction fingerprints from protein molecular surfaces using geometric deep learning (2019)*](https://www.nature.com/articles/s41592-019-0666-6).

$1$. In 2020 we made a huge [step forward](https://deepmind.com/blog/article/alphafold-a-solution-to-a-50-year-old-grand-challenge-in-biology) in the problem of protein folding thanks to deep learning. Can we say that protein folding has been solved by AlphaFold 2? [Not quite](http://backreaction.blogspot.com/2021/01/has-protein-folding-been-solved.html).



## Digesting *a priori* the information in the structure

We have already learned how to consider the structure information in a simple case; when the domain $\Omega$ is the Euclidean $\mathbb{R}^2$, i.e. when the relations between variables are represented by a 2-dimensional grid, such as the pixels of an image.

![immagine griglia 2d](https://raw.githubusercontent.com/lucmos/DLAI-s2-2020-tutorials/master/09/pics/euclidean.PNG)

The filters of a convolutional neural network elaborate pixels considering only their neighbors and are applied with the same weights all over the image. In this way **CNNs are digesting *a priori* the structure information**, using the properties of the domain -- the translational invariance and the locality of neighbours -- to **reduce the free parameters of the model**. This leads to a crucial speed-up of the training process and allows larger and more powerful models.

CNNs can be naturally extended to general Euclidean domains $\mathbb{R}^n$, but what can we do when $\Omega \neq \mathbb{R}^n$?

In many different fields such as Biology, Physics, Social Sciences and Computer Graphics we have to process signals defined on non-Euclidean domains, such as Graphs $\Omega = G(\mathcal{V, E})$ or [Manifolds](https://en.wikipedia.org/wiki/Manifold) $\Omega = \mathcal{X}$.

We would like to come out with a solution analoguous to CNNs; digesting *a priori* the structure information to reduce the free parameters proved to be very convenient in a learning setting based on a gradient descent optimization.

### Representing non-Euclidean data in an Euclidean memory

Working with non-Euclidean domains provides a further challenge in representing the data.

In the Euclidean setting, encoding the data in ordered matrices, vectors or tensors is so natural and effective that we do not even think about alternatives, and indeed the computer memory structure is itself Euclidean.

In the non-Euclidean setting we have many alternatives, consider for instance a manifold $\mathcal{X}$. It can be represented by a triangle mesh with vertices, edges and faces $\mathcal{V,E,F}$, by a n-polygonal mesh, where we admit also non-triangle faces, by a simple point cloud or by a subdivision surface.

And even when we have chosen the representation, we still have to come out with a last encoding procedure to store our data in matrices and tensors as required by the Euclidean structure of the physical memory in our computers.

## GCN & CORA

The code in the following sections comes mainly from [this repository](https://github.com/tkipf/pygcn) and it is inspired by the paper [Semi-Supervised Classification with Graph Convolutional Networks](https://arxiv.org/abs/1609.02907), where Thomas Kipf presents the Graph Convolutional Networks in PyTorch.


### The CORA dataset

The CORA dataset is a much bigger dataset when compared to the Karate graph, the task is again node classification.

In the CORA graph we have:
- Each **node** is a Machine Learning paper.
- An **edge** represents one citation from one paper to another.
- Each node is classified into one of seven possible Machine Learning sub-fields:
 - Case Based
 - Genetic Algorithms
 - Neural Networks
 - Probabilistic Methods
 - Reinforcement Learning
 - Rule Learning
 - Theory

We will use a subset of the CORA dataset, preprocessed as suggested by [Thomas Kipf](https://github.com/tkipf/pygcn/tree/master/data/cora):

- It considers only papers that are cited or cite at least once.
- The words are [stemmed](https://en.wikipedia.org/wiki/Stemming)
- Stopwords and infrequent words are removed.

This subset contains **2708** paper with **1433** unique words.


---

[Here](http://networkrepository.com/cora.php) you can have fun exploring the complete CORA dataset, and many other graph datasets.

In [ ]:
classes = [
    "Case_Based",
    "Genetic_Algorithms",
    "Neural_Networks",
    "Probabilistic_Methods",
    "Reinforcement_Learning",
    "Rule_Learning",
    "Theory",
]

### Dataset format

In [ ]:
# The preprocessed CORA is contained in this repository under ./data/cora
!git clone https://github.com/tkipf/pygcn.git

The directory `pygnc/data/cora` contains two files: `cora.content` and `cora.cites`.

The `cora.content` contains the description of each node. For each line it contains:
 - The id of the node.
 - The [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model) text representation.
 - The label of that node.

In [ ]:
import pandas

headers = ["PaperID"] + [f"word{i}" for i in range(1433)] + ["label"]
pandas.read_csv("pygcn/data/cora/cora.content", sep="\t", names=headers)

The `cora.cites` contains the relationships between nodes. For each line it contains:

- The first entry is id of the cited paper
- The second entry id of the citing paper

That is, the direction of the entry is right to left.

In [ ]:
import pandas

headers = ["Cited PaperID", "Citing PaperID"]
pandas.read_csv("pygcn/data/cora/cora.cites", sep="\t", names=headers)

### Data loading

The repository provides python functions to parse the preprocessed data.

It returns the adjacency matrix, the node features, the labels for each node, the indices to split into train-test:


In [ ]:
# Add the folder to the python path
import sys

sys.path.insert(0, "./pygcn/pygcn")

from utils import load_data

adj, features, labels, idx_train, idx_val, idx_test = load_data(path="pygcn/data/cora/")

In [ ]:
# As expected its shape is (num_paper, num_paper)
adj.shape

In [ ]:
# Each paper has 1433 features. The Bag Of Words representation of its text (i.e., the set of words in the text)
features.shape

In [ ]:
# @title utility functions


def get_predictions(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    return correct


def accuracy(output, labels):
    correct = get_predictions(output, labels)
    correct = correct.sum()
    return correct / len(labels)


def plot_loss(losses):
    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=list(range(len(losses))),
            y=losses,
            # name="Name of Trace 1"       # this sets its legend entry
        )
    )

    fig.update_layout(
        title="Train loss",
        xaxis_title="Epoch",
        yaxis_title="Loss",
        font=dict(family="Courier New, monospace", size=18, color="#7f7f7f"),
    )
    return fig


def refresh_bar(bar, desc):
    bar.set_description(desc)
    bar.refresh()


import plotly as py
import plotly.graph_objs as go
import igraph


def plot_graph(adj, node_colors, colors_legend=classes, title="CORA graph", layt=None):
    N = adj.shape[0]
    adj = adj.coalesce()
    edgeA, edgeB = adj.indices()[0, :], adj.indices()[1, :]
    edgeA = edgeA.tolist()
    edgeB = edgeB.tolist()

    G = igraph.Graph.Adjacency((adj.to_dense() > 0).tolist())
    if layt is None:
        layt = G.layout("fr_3d")

    Xn = [layt[k][0] for k in range(N)]  # x-coordinates of nodes
    Yn = [layt[k][1] for k in range(N)]  # y-coordinates
    Zn = [layt[k][2] for k in range(N)]  # z-coordinates
    Xe = []
    Ye = []
    Ze = []
    for e in zip(edgeA, edgeB):
        Xe += [layt[e[0]][0], layt[e[1]][0], None]  # x-coordinates of edge ends
        Ye += [layt[e[0]][1], layt[e[1]][1], None]
        Ze += [layt[e[0]][2], layt[e[1]][2], None]

    trace1 = go.Scatter3d(
        x=Xe,
        y=Ye,
        z=Ze,
        mode="lines",
        line=dict(color="rgb(125,125,125)", width=1),
        hoverinfo="none",
    )

    trace2 = go.Scatter3d(
        x=Xn,
        y=Yn,
        z=Zn,
        mode="markers",
        name="actors",
        marker=dict(
            symbol="circle",
            size=6,
            color=node_colors,
            colorscale="Viridis",
            line=dict(color="rgb(50,50,50)", width=0.5),
        ),
        text=colors_legend,
        hoverinfo="text",
    )

    axis = dict(
        showbackground=False,
        showline=False,
        zeroline=False,
        showgrid=False,
        showticklabels=False,
        title="",
    )

    layout = go.Layout(
        title=title,
        width=800,
        height=800,
        showlegend=False,
        scene=dict(
            xaxis=dict(axis),
            yaxis=dict(axis),
            zaxis=dict(axis),
        ),
        margin=dict(t=100),
        hovermode="closest",
    )

    data = [trace1, trace2]
    fig = go.Figure(data=data, layout=layout)
    return fig, layt

### CORA Graph

Let's visualize the graph!

As you can see, the graph has many disconnected components, many of which are very small. The preprocessing ensures that each component has at least 2 nodes.


Note that in the visualization, nodes with the same colors have the same label.


In [ ]:
fig, layt = plot_graph(adj, labels)
fig

### MLP approach

The simplest approach is to use a Multi Layer Perceptron on the features of each node, independently.

This means that we aim to predict the sub-field of each machine learning paper looking exclusively at its text, encoded in a BOW. We're not considering at all at the structure of the graph, i.e. the citations that link the papers.

You can see this approach as using only the **signal** part of the data, ignoring the **structure**.

We can draw a parallel with a task in the image domain: predicting if a pixel belongs to a part of the image where there is sky or not. This MLP approach would independently process each pixel in an image, without looking at its context.

In [ ]:
def mlp_accuracy(model):
    """
    Perfom a forward pass `y_pred = model(x)` and computes the accuracy
    between `y_pred` and `y_true`
    """
    model.eval()
    y_pred = model(features[idx_test])
    acc = accuracy(y_pred, labels[idx_test])
    print(f"Accuracy: {acc:.5}")
    return acc

In [ ]:
# Model definition
mlp = nn.Sequential(
    nn.Linear(1433, 500), nn.ReLU(), nn.Linear(500, 100), nn.ReLU(), nn.Linear(100, 7)
)

In [ ]:
print("[MLP] before training")
_ = mlp_accuracy(mlp)
print(
    f"Which is comparable to/worse than random guessing (there are 7 classes in total): {1/7=:.5}"
)

Cora graph visualization:
- **Yellow nodes: correct predictions**
- **Purple nodes: wrong predictions**

In [ ]:
correct = get_predictions(mlp(features), labels)
fig, layt = plot_graph(adj, correct, title="MLP performance before training", layt=layt)
fig

In [ ]:
# Let's now train the MLP model
opt = optim.Adam(mlp.parameters())

losses = []
mlp.train()

for epoch in trange(500):
    opt.zero_grad()
    output = mlp(features[idx_train])
    loss = F.cross_entropy(output, labels[idx_train])  # train only on the train samples
    loss.backward()
    opt.step()

    losses.append(loss.item())

plot_loss(losses)

In [ ]:
print("[MLP] after training")
accmlp = mlp_accuracy(mlp)

Cora graph visualization:
- Yellow nodes: correct predictions
- Purple nodes: wrong predictions

In [ ]:
correct = get_predictions(mlp(features), labels)
fig, layt = plot_graph(adj, correct, title="MLP performance after training", layt=layt)
fig

### Graph convolutional network


We can define an equivalent of the `nn.Layer` that uses the adjacency matrix (and therefore the data structure) in the forward pass:

In [ ]:
from torch.nn import Parameter

import math


class GraphConvolution(Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolution, self).__init__()

        # A nn.Parameter is a normal tensor
        # that is automatically registered as a model parameter
        # so that it is inclued in `model.parameters()`.
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        self.bias = Parameter(torch.FloatTensor(out_features))
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        self.bias.data.uniform_(-stdv, stdv)

    def forward(self, x, adj):
        support = torch.mm(x, self.weight)
        output = torch.spmm(adj, support)  # sparse matrix multiplication
        # note that we are using the WHOLE graph to compute the output since we are using the adjacency matrix
        return output + self.bias

These layers can be combined together to build complex models

In [ ]:
class GCN(nn.Module):
    def __init__(self, input_features: int, hidden_dim: int, num_classes: int):
        super(GCN, self).__init__()
        self.gc1 = GraphConvolution(input_features, hidden_dim)
        self.gc2 = GraphConvolution(hidden_dim, num_classes)

    def forward(self, x, adj):
        x = self.gc1(x, adj)
        x = F.relu(x)
        x = self.gc2(x, adj)
        return x

In [ ]:
def gcn_accuracy(model):
    """
    Perfom a forward pass `y_pred = model(x)` and computes the accuracy
    between `y_pred` and `y_true`.

    It is particuarly tricky to perform batching in GCN.
    As you can see, here the forward pass is performed on the whole graph
    """
    model.eval()
    y_pred = model(features, adj)  # Do you notice the difference?
    acc = accuracy(y_pred[idx_test], labels[idx_test])
    print(f"Accuracy: {acc:.5}")
    return acc

In [ ]:
gcn = GCN(1433, 50, 7)

print("[GCN] before training")
_ = gcn_accuracy(gcn)
print(
    f"Which is comparable to/worse than random guessing (there are 7 classes in total): {1/7=:.5}"
)

Cora graph visualization:
- Yellow nodes: correct predictions
- Purple nodes: wrong predictions

In [ ]:
correct = get_predictions(gcn(features, adj), labels)
fig, layt = plot_graph(adj, correct, title="GCN performance before training", layt=layt)
fig

In [ ]:
# Let's now train the GCN model
opt = optim.Adam(gcn.parameters())

losses = []
gcn.train()

for epoch in trange(1000):
    opt.zero_grad()
    output = gcn(
        features, adj
    )  # compute all outputs, even for the nodes in the test set
    loss = F.cross_entropy(
        output[idx_train], labels[idx_train]
    )  # Train only on the train samples!
    loss.backward()
    opt.step()

    losses.append(loss.item())

plot_loss(losses)

In [ ]:
print("[GCN] after training")
accgcn = gcn_accuracy(gcn)

Cora graph visualization:
- Yellow nodes: correct predictions
- Purple nodes: wrong predictions

In [ ]:
correct = get_predictions(gcn(features, adj), labels)
fig, layt = plot_graph(adj, correct, title="GCN performance after training", layt=layt)
fig

In [ ]:
def num_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print("Number of parameters MLP: ", num_params(mlp))
print("Number of parameters GCN: ", num_params(gcn))

### Performance comparison

In [ ]:
fig = go.Figure([go.Bar(x=["MLP", "GCN"], y=[accmlp.item(), accgcn.item()])])
fig.update_layout(
    title="Performance comparison", yaxis_title="Accuracy [%]", xaxis_title="Model type"
)
fig.show()

**Credits**

- Geometric deep learning [tutorial](https://vistalab-technion.github.io/cs236781/tutorials/tutorial_09/)
- Deep Learning and Applied AI course at Sapienza, [GNN lesson](https://github.com/erodola/DLAI-s2-2023)
- Kipf T, Welling M. Semi-Supervised Classification with Graph Convolutional Networks (2016).
- Bronstein M. M., et al. (2017) Geometric Deep Learning: Going beyond Euclidean data. IEEE Signal Process Mag 34(4).